# Load, train and run (predict) a DNN

First, we are going to look in detail the process, and then we are going to show the function for a full train

In [1]:
import torch
from argparse     import ArgumentParser
from argparse     import Namespace

In [2]:
import numpy  as np
import pandas as pd
import tables as tb

In [3]:
import invisible_cities.io.dst_io as dio

In [4]:
from next_sparseconvnet.utils.data_loaders     import LabelType,       DataGen, collatefn, weights_loss
from next_sparseconvnet.networks.architectures import NetArchitecture, UNet, ResNet
from next_sparseconvnet.utils.train_utils      import *
from next_sparseconvnet.utils.focal_loss       import FocalLoss

## Load data

Take the path of the train and validation files we want to use.

In [5]:
train_path = "/lustre/ific.uv.es/ml/ific020/nexus_2021/train_dataset_5mm_all.h5"
valid_path = "/lustre/ific.uv.es/ml/ific020/nexus_2021/valid_dataset_5mm.h5"

We can explore how do the files look like. With the new labelling files may have some differences, but not many.

In [6]:
with tb.open_file(train_path, 'r') as h5in:
    print(h5in)

/lustre/ific.uv.es/ml/ific020/nexus_2021/train_dataset_5mm_all.h5 (File) ''
Last modif.: 'Wed Jun  2 15:47:58 2021'
Object Tree: 
/ (RootGroup) ''
/DATASET (Group) ''
/DATASET/BinsInfo (Table(1,), shuffle, zlib(4)) ''
/DATASET/EventsInfo (Table(673706,), shuffle, zlib(4)) ''
/DATASET/Voxels (Table(34817578,), shuffle, zlib(4)) ''



First we have the bin information of the data, so we can see how the detector is discretized. This data is for NEXT-White.

In [7]:
dio.load_dst(train_path, 'DATASET', 'BinsInfo')

,min_x,max_x,nbins_x,min_y,max_y,nbins_y,min_z,max_z,nbins_z,Rmax
0,-220,220,89,-220,220,89,0,550,111,220


Then we have a mapping between the events here contained and the files where each event came from. 

Originally, the MC events were distributed within several files (basename) with a specific identificator (event_id), and when we prepared the dataset, we joint all of them so we gave them a new unique identifyer (dataset_id), as event_id was repeated along different files. 

Also, it has per event the binary class (binclass) information, that tells us whether the event is background-0, or signal-1. 

The pathname was the rest of the directory name for the MC files, but in gpu1. In artemisa, the pathname is '/lustre/ific.uv.es/ml/ific020/nexus_2021/MC_production'. Having the MC information is useful to check what happened in an specific event.

In [8]:
dio.load_dst(train_path, 'DATASET', 'EventsInfo')

,event_id,binclass,pathname,basename,dataset_id
0,100000000,0,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_100_tlde.h5,0
1,100000001,0,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_100_tlde.h5,1
2,100000002,1,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_100_tlde.h5,2
3,100000003,0,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_100_tlde.h5,3
4,100000004,0,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_100_tlde.h5,4
...,...,...,...,...,...
673701,1999000337,1,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_1999_tlde.h5,644240
673702,1999000338,0,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_1999_tlde.h5,644241
673703,1999000339,0,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_1999_tlde.h5,644242
673704,1999000340,1,/home/gdiaz/data_for_marija/tlde_nn/nexus,nexus_1999_tlde.h5,644243


Finally we have the voxels for each event, with position, energy and the segmentation class (segclass), which is 0-other particles, 1-track, 2-blob

In [9]:
dio.load_dst(train_path, 'DATASET', 'Voxels')

,xbin,ybin,zbin,energy,segclass,binclass,dataset_id
0,47,45,74,0.028918,1,0,0
1,43,48,78,0.043978,1,0,0
2,41,45,81,0.012179,1,0,0
3,49,46,74,0.005277,1,0,0
4,43,48,77,0.009145,1,0,0
...,...,...,...,...,...,...,...
34817573,19,33,68,0.049901,1,0,644244
34817574,19,31,68,0.018297,1,0,644244
34817575,19,33,69,0.005991,1,0,644244
34817576,16,35,70,0.032205,1,0,644244


We now choose one of the two ways to read the data (either for classification or for segmentation) and the amount of data we want to load. Also, we can decide whether or not to perform data augmentation (rotating and mirroring data randomly)

In [10]:
label_type = LabelType.Classification
#label_type = LabelType.Segmentation

#we use few data as it is a test, more data would become very heavy
nevents_train = 100 
nevents_valid = 10

augmentation = True

And create the Data Generator for train and validation

In [11]:
datagen_train = DataGen(train_path, label_type, nevents = nevents_train, augmentation = augmentation)
datagen_valid = DataGen(valid_path, label_type, nevents = nevents_valid)

The Data Generator stores the data in the way that the NN needs. As we see in the next cell, picking an element returns a tuple with arrays (xbin, ybin, zbin, energy, binclass/segclass, dataset_id)

In [12]:
datagen_train[14]

(array([53, 68, 50, 54, 57, 49, 50, 56, 53, 52, 50, 54, 50, 83, 52, 60, 57,
        60, 57, 50, 55, 51, 49, 53, 51, 51, 60, 52, 53, 52, 51, 51, 60, 60,
        52, 52, 68, 60, 57, 57, 53, 51, 60, 50, 51, 52, 58, 59]),
 array([22, 11, 21, 22, 19, 22, 22, 19, 19, 19, 22, 19, 22, 48, 19, 20, 20,
        20, 20, 21, 19, 22, 22, 21, 20, 20, 21, 20, 21, 22, 21, 21, 22, 21,
        20, 18, 10, 19, 19, 19, 18, 22, 22, 20, 22, 18, 19, 19]),
 array([85, 94, 82, 85, 86, 83, 84, 86, 86, 84, 83, 86, 82, 70, 80, 89, 87,
        88, 86, 81, 86, 81, 82, 84, 80, 81, 88, 80, 85, 84, 82, 81, 89, 89,
        84, 84, 84, 89, 87, 88, 85, 82, 88, 81, 84, 85, 89, 89]),
 array([0.03313701, 0.0343953 , 0.12476786, 0.00629492, 0.02176605,
        0.01348927, 0.01821891, 0.03875297, 0.02572587, 0.03190605,
        0.02469233, 0.02831793, 0.021687  , 0.0018993 , 0.0084107 ,
        0.03359486, 0.0260051 , 0.01074126, 0.0081438 , 0.03462485,
        0.03355365, 0.04162054, 0.00458728, 0.00430069, 0.09175134,
      

Now we also pick the size of the batch. A batch is a group of events that is passed through the NN together; it is a common practice to divide all the data into these groups.

In [13]:
batch_size_train = 20
batch_size_valid = 2

And create the Data Loader, which is the object that will pass through the net with all the information transformed into torch tensors

In [14]:
loader_train = torch.utils.data.DataLoader(datagen_train, batch_size = batch_size_train, shuffle = True, num_workers=1, collate_fn=collatefn, drop_last=True, pin_memory=False)
loader_valid = torch.utils.data.DataLoader(datagen_valid, batch_size = batch_size_valid, shuffle = True, num_workers=1, collate_fn=collatefn, drop_last=True, pin_memory=False)

For example, with 100 events and a batch size of 20, we will have 5 batches. 

If we have n_voxels for N events, the batch will be made of a tuple with size [n_voxels, 4] for the coordinates (xbin, ybin, zbin, id); [n_voxels, 1] for the energy; [N] or [n_voxels] with the binclass or segmentation and [N] with the dataset_id.

In [15]:
for batch in loader_train:
    print(batch[0].size(), batch[1].size(), batch[2].size(), batch[3].size())

torch.Size([1037, 4]) torch.Size([1037, 1]) torch.Size([20]) torch.Size([20])
torch.Size([1095, 4]) torch.Size([1095, 1]) torch.Size([20]) torch.Size([20])
torch.Size([1039, 4]) torch.Size([1039, 1]) torch.Size([20]) torch.Size([20])
torch.Size([977, 4]) torch.Size([977, 1]) torch.Size([20]) torch.Size([20])
torch.Size([1057, 4]) torch.Size([1057, 1]) torch.Size([20]) torch.Size([20])


We can see here the batch's dataset_ids, which are randomly picked using the parametre shuffle in the Data Loader.

In [16]:
batch[3]

tensor([68, 60, 70, 36, 44,  9, 80, 37, 71, 74, 35, 11, 73, 94, 46, 57, 99, 85,
        25, 51], dtype=torch.int32)

As a summary, the needed parameters for the loading of the data are:

* label type (Classification/Segmentation)
* number of events
* batch size
* augmentation

## Create the net

Then, we create the net as an object; we have two different nets, but as we know, the major part of the structure is similar, so the needed parameters will be pretty much the same excepting a couple.

In [17]:
#COMMON NET PARAMS
spatial_size      = (543, 543, 543)
init_conv_nplanes = 8
init_conv_kernel  = 7
kernel_sizes      = [7, 7, 5, 3, 3, 3]
stride_sizes      = [4, 2, 2, 2, 2]
basic_num         = 2
momentum          = 0.7

#RESNET
nlinear           = 32

#UNET
nclasses          = 3

The explanation of the parameters is found in the tutorial.

We load the net simply as:

In [18]:
#CLASSIFICATION
net = ResNet(spatial_size,
             init_conv_nplanes,
             init_conv_kernel,
             kernel_sizes,
             stride_sizes,
             basic_num,
             momentum = momentum,
             nlinear  = nlinear)

In [19]:
#SEGMENTATION
#net = UNet(spatial_size,
#           init_conv_nplanes,
#           init_conv_kernel,
#           kernel_sizes,
#           stride_sizes,
#           basic_num,
#           momentum = momentum, 
#           nclasses = nclasses)

In the practice, the architecture is chosen usingn the netarch param, and it can be either NetArchitecture.ResNet or NetArchitecture.UNet

We dont use cuda in the notebook because i couldn't make it work, i think because artemisa limits the gpu cuda usage to only sending jobs or trying them with an interactive session of 5 minutes

In [20]:
#net = net.cuda()

We can see the layers and configuration of the net

In [21]:
net

ResNet(
  (inp): InputLayer()
  (convBN): ConvBNBlock(
    (conv): SubmanifoldConvolution 1->8 C7
    (bnr): BatchNormReLU(8,eps=0.0001,momentum=0.7,affine=True)
  )
  (downsample): ModuleList(
    (0): ResidualBlock_downsample(
      (bnr1): BatchNormReLU(8,eps=0.0001,momentum=0.7,affine=True)
      (conv1): Convolution 8->16 C7/4
      (bnr2): BatchNormReLU(16,eps=0.0001,momentum=0.7,affine=True)
      (subconv): SubmanifoldConvolution 16->16 C7
      (conv2): Convolution 8->16 C7/4
      (add): AddTable()
    )
    (1): ResidualBlock_downsample(
      (bnr1): BatchNormReLU(16,eps=0.0001,momentum=0.7,affine=True)
      (conv1): Convolution 16->32 C7/2
      (bnr2): BatchNormReLU(32,eps=0.0001,momentum=0.7,affine=True)
      (subconv): SubmanifoldConvolution 32->32 C7
      (conv2): Convolution 16->32 C7/2
      (add): AddTable()
    )
    (2): ResidualBlock_downsample(
      (bnr1): BatchNormReLU(32,eps=0.0001,momentum=0.7,affine=True)
      (conv1): Convolution 32->64 C5/2
      (bn

## Choose and configure CRITERION and OPTIMIZER

The criterion quantifies how good the NN is performing on the fly. We have used the Cross Entropy Loss, which is a well extended loss function, and also the Focal Loss, which gave very good results for the semantic segmentation issue, as it is specific for segmentation problems.

The optimizer updates the weights of the different layers in the NN, searching for a minimum in the loss function.

### CRITERION

The criterion will require some weights to give more value to some classes than others. For Classification, we need 2 weights as there are 2 classes, and for Segmentation 3. 

In the configs there are two parameters related to this:
* LossType - can be 'CrossEntropyLoss' or 'FocalLoss' (the latter is more suitable for Segmentation)
* weights_loss - if None, there are no weights used; if True, weights are calculated using inverse frequencies of each class in the data; also a list of values can be specified. For the Focal Loss these weights are a parameter alpha.

For example, for the label type chosen before in the notebook, we can compute with the following function those weights

In [22]:
weights_loss_values = weights_loss(train_path, 10000, label_type, effective_number=False)
print(weights_loss_values)

[0.71442885 0.28557115]


And we can create our criterion; if we chose CrossEntropyLoss:

In [23]:
criterion = torch.nn.CrossEntropyLoss(weight = torch.Tensor(weights_loss_values))

If we are using Segmentation and chose FocalLoss, this loss was implemented by hand (Cross Entropy Loss comes with torch) and the optimal parameters, as in the paper (arXiv:1708.02002) would be: alpha = 0.25 (per class) and gamma = 2

In [24]:
#This would be for segmentation, as alpha is a list with 3 elements.
#criterion = FocalLoss(alpha=[0.25, 0.25, 0.25], gamma=2.)

### OPTIMIZER

The optimizer will require the following parameters:

In [25]:
lr = 1e-2
betas = (0.9, 0.999)
eps = 1e-6
weight_decay = 0

In [26]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),
                                     lr = lr,
                                     betas = betas,
                                     eps = eps,
                                     weight_decay = weight_decay)

## TRAIN&VALID ONE EPOCH

Now that we have the data loaded, the net built, the criterion and the optimizer chosen and configured, we can train the net. 

The following function trains the net passing through all the used data once: this is an epoch. In a full training, you usually train for several epoches, and each time the net is expected to have better results. 

This function prints the number of the epoch, and the loss computed with the chosen criterion, and also returns the value of the loss and the value of the computed metric. For Classificacion, we use the regular accuracy, and for Segmentation we use IoU.

I've added the False use_cuda argument because as I said, I couldn't get cuda to work in jupyter notebook

In [27]:
train_one_epoch(0, net, criterion, optimizer, loader_train, label_type, use_cuda = False)

#the warning is for something that SparseConvNet hasn't updated and PyTorch complains

/lustre/ific.uv.es/ml/ific020/software/SparseConvNet/sparseconvnet/convolution.py:36: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  (input.spatial_size - self.filter_size) // self.filter_stride + 1
/lustre/ific.uv.es/ml/ific020/software/SparseConvNet/sparseconvnet/maxPooling.py:83: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input.s

Train Epoch: 0	 Loss: 1.456046


(1.4560464382171632, tensor(0.6400))

During the training is also important to cross check the performance of the net with some different data o avoid over training, so there is an analogue function for validation, which is used right after each epoch train. The main difference between them is that in the train function, the net weights are updated and the net ''improves'', while for the validation function we just evaluate the net and compute the loss etc, without using the optimizer!

In [28]:
valid_one_epoch(net, criterion, loader_train, label_type, use_cuda = False)

	 Validation Loss: 4.260647


(4.26064670085907, tensor(0.7100))

## TRAIN NET COMLPETELY

So, the train_net function used in the main.py script what does is, for a certain number of epoches:
* Loads data
* Trains and validates an epoch of the loaded data with the net, criterion and optimizer chosen
* Saves a checkpoint of the net (similar to a photograph of the current status) if the validation loss improved with respect to the previous epoch. If the loss of an epoch starts raising it is an indicator of overtraining. This checkpoint is used either to continue a train starting from an already trained net, or to use the net to get results!
* Saves the metrics and loss for train and validation data, to explore the performance of the whole train

Then, we need some new parameters:
* nepoch - the number of epochs we want to train
* checkpoint_dir - directory to save the checkpoints
* tensorboard_dir - directory to save the metrics and loss (there is a notebook called read_tensorboard_artemisa which explains how to read these files; then also there is an interactive option using in artemisa the command ''tensorboard --logdir=/path/to/tensorboard/file''
* num_workers

I created a folder in examples to store the checkpoints and tensorboards

In [29]:
nepoch = 3
checkpoint_dir = '/lhome/ext/ific020/ific0201/NEXT_SPARSECONVNET/examples/net_savings/checkpoints'
tensorboard_dir = '/lhome/ext/ific020/ific0201/NEXT_SPARSECONVNET/examples/net_savings/logs'
num_workers = 1

In [30]:
train_net(nepoch = nepoch,
          train_data_path = train_path,
          valid_data_path = valid_path,
          train_batch_size = batch_size_train,
          valid_batch_size = batch_size_valid,
          net = net,
          label_type = label_type,
          criterion = criterion,
          optimizer = optimizer,
          checkpoint_dir = checkpoint_dir,
          tensorboard_dir = tensorboard_dir,
          num_workers = num_workers,
          nevents_train = nevents_train,
          nevents_valid = nevents_valid,
          augmentation  = augmentation, 
          use_cuda = False)

/lustre/ific.uv.es/ml/ific020/software/SparseConvNet/sparseconvnet/convolution.py:36: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  (input.spatial_size - self.filter_size) // self.filter_stride + 1
/lustre/ific.uv.es/ml/ific020/software/SparseConvNet/sparseconvnet/maxPooling.py:83: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input.s

Train Epoch: 0	 Loss: 0.724674
	 Validation Loss: 0.632538


/lustre/ific.uv.es/ml/ific020/software/miniconda/envs/IC-3.7-2020-06-16/lib/python3.7/site-packages/torch/nn/modules/module.py:1384: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  " and ".join(warn_msg) + " are deprecated. nn.Module.state_dict will not accept them in the future. "


Train Epoch: 1	 Loss: 0.575022
	 Validation Loss: 0.571277
Train Epoch: 2	 Loss: 0.731007
	 Validation Loss: 0.530173


Now we have a trained net, after some epochs! We can choose to keep on training it more epochs, just passing the net through the function again, changing things trying to improve, or we can use it now to predict!

# PREDICT WITH THE NET

As we have an open net, we can use it directly in the function predict_gen, but we can also create a net loading one of the checkpoints if we jumped the cells in 'TRAIN&VALID ONE EPOCH' and 'TRAIN NET COMLPETELY' sections but we have saved checkpoints from other day. For example in my case would be:

In [45]:
saved_weights = '/lhome/ext/ific020/ific0201/NEXT_SPARSECONVNET/examples/net_savings/checkpoints/net_checkpoint_2.pth.tar'

#uncommment this to read one of your saved checkpoints

#dct_weights = torch.load(saved_weights)['state_dict']
#net.load_state_dict(dct_weights, strict=False)
#print('weights loaded')

For the prediction we choose the file to predict (usually is the validation file), the batch size for performin the prediction and the number of events to predict. I will use all the same as for the training.

In [42]:
gen = predict_gen(data_path = valid_path,
                  label_type = label_type,
                  net = net,
                  batch_size = batch_size_valid,
                  nevents = nevents_valid, 
                  use_cuda = False)

In [43]:
for dct in gen:
    print(dct)

{'label': array([0, 0]), 'dataset_id': tensor([0, 1], dtype=torch.int32), 'prediction': array([[0.7631363 , 0.23686369],
       [0.69393784, 0.30606216]], dtype=float32)}
{'label': array([0, 0]), 'dataset_id': tensor([2, 3], dtype=torch.int32), 'prediction': array([[0.61775494, 0.38224503],
       [0.6239486 , 0.37605146]], dtype=float32)}
{'label': array([0, 1]), 'dataset_id': tensor([4, 5], dtype=torch.int32), 'prediction': array([[0.6939479 , 0.30605212],
       [0.6611927 , 0.3388073 ]], dtype=float32)}
{'label': array([1, 0]), 'dataset_id': tensor([6, 7], dtype=torch.int32), 'prediction': array([[0.6937557 , 0.30624425],
       [0.6496358 , 0.3503642 ]], dtype=float32)}
{'label': array([1, 0]), 'dataset_id': tensor([8, 9], dtype=torch.int32), 'prediction': array([[0.729201 , 0.270799 ],
       [0.7206646, 0.2793354]], dtype=float32)}


As we have 10 events in batches of 2 events, we will have 5 batches. If we look to what each batch returns, we see that we have the label of the event, the dataset_id of the event, and the prediction for each event

In [44]:
dct

{'label': array([1, 0]),
 'dataset_id': tensor([8, 9], dtype=torch.int32),
 'prediction': array([[0.729201 , 0.270799 ],
        [0.7206646, 0.2793354]], dtype=float32)}

For example, the event 8 was signal (label 1) and have a prediction of [0.729201 , 0.270799 ], which means that it has a probability of 73% to be class 0 and 27% to be class 1.

For the segnet, the main difference would be that in the prediction we will have 3 values, as we have 3 classes, and the prediction will be done FOR EACH VOXEL of the events in the batch, so we would also have the information of the coordinates of the voxels

The rest of the main script is for arranging these results in a dataframe and writing it into a new file, defined in the out_file parameter